In [1]:
pip install selenium pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [21]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1) 크롬 실행
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 2) 네이버 지도 접속
driver.get("https://map.naver.com/p")
time.sleep(2)

# 3) 검색
query = "서울 성동구 무학로 10-1 베이커리"
search_box = driver.find_element(By.CLASS_NAME, "input_search")
search_box.send_keys(query)
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# 4) 첫 번째 결과 클릭
first_result = driver.find_element(By.CSS_SELECTOR, "button.link_space")
first_result.click()
time.sleep(3)

# 5) iframe 전환 (상세 페이지는 entryIframe 안에 로딩됨)
driver.switch_to.frame("entryIframe")

# 6) 리뷰 탭 클릭
wait = WebDriverWait(driver, 15)
review_tab = wait.until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(@class,'_tab-menu')]//span[text()='리뷰']"))
)
review_tab.click()
time.sleep(3)

# 7) 첫 번째 리뷰 li 기다림
first_li = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "li.place_apply_pui"))
)

# 8) 리뷰 텍스트 추출
review_text = first_li.find_element(By.CSS_SELECTOR, "div.pui__vn15t2 a").text
print("첫 번째 리뷰 텍스트:", review_text)

# 9) DataFrame에 담기
df = pd.DataFrame([{"review": review_text}])
print(df)

# driver.quit()  # 다 끝나면 크롬 닫기


첫 번째 리뷰 텍스트: 2호선을 이용할 때면 상왕십리역에서 내려 버스를 타곤 하는데, 그럴 때마다 눈에 쏘-옥 들어왔던 매장입니다-🍰

때마침 집에 가는 길에 여자친구가 달다구리 케이크를 먹고 싶다 해서 오예! 하고 방문했네용ㅎㅎ 아직 맛은 보지 않았으나 후기들만 봐도 평이 정말 좋아 벌써 만족스럽네요 크크- 맛있으면 자주 방문할게요 사장님 :)
                                              review
0  2호선을 이용할 때면 상왕십리역에서 내려 버스를 타곤 하는데, 그럴 때마다 눈에 쏘...


In [23]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ===== 설정 =====
# 실제 알고 있는 가게 이름 (모자이크 되기 전 원본)
target_name = "케이크 달달"   # 예시: 실제 상호명
target_prefix = target_name[:2]   # 앞 두 글자
target_length = len(target_name)  # 전체 길이

# ===== 크롬 실행 =====
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 1) 네이버 지도 접속
driver.get("https://map.naver.com/p")
time.sleep(2)

# 2) 검색
query = "서울 성동구 무학로 10-1 베이커리"
search_box = driver.find_element(By.CLASS_NAME, "input_search")
search_box.send_keys(query)
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# 3) 검색 결과 리스트 확인
results = driver.find_elements(By.CSS_SELECTOR, "button.link_space")

matched = None
for r in results:
    try:
        name = r.find_element(By.CSS_SELECTOR, "strong.space_title").text.strip()
        if name.startswith(target_prefix) and len(name) == target_length:
            matched = r
            print("✅ 일치하는 가게 찾음:", name)
            break
    except:
        continue

if matched is None:
    print("❌ 조건에 맞는 가게를 찾지 못했습니다.")
    driver.quit()
else:
    # 4) 해당 가게 클릭
    matched.click()
    time.sleep(3)

    # 5) iframe 전환 (상세 페이지는 entryIframe 안에 로딩됨)
    driver.switch_to.frame("entryIframe")

    # 6) 리뷰 탭 클릭
    wait = WebDriverWait(driver, 15)
    review_tab = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(@class,'_tab-menu')]//span[text()='리뷰']"))
    )
    review_tab.click()
    time.sleep(3)

    # 7) 첫 번째 리뷰 li 기다림
    first_li = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li.place_apply_pui"))
    )

    # 8) 리뷰 텍스트 추출
    review_text = first_li.find_element(By.CSS_SELECTOR, "div.pui__vn15t2 a").text
    print("첫 번째 리뷰 텍스트:", review_text)

    # 9) DataFrame에 담기
    df = pd.DataFrame([{"review": review_text}])
    print(df)

    # driver.quit()  # 끝나면 크롬 닫기


✅ 일치하는 가게 찾음: 케이크 달달
첫 번째 리뷰 텍스트: 2호선을 이용할 때면 상왕십리역에서 내려 버스를 타곤 하는데, 그럴 때마다 눈에 쏘-옥 들어왔던 매장입니다-🍰

때마침 집에 가는 길에 여자친구가 달다구리 케이크를 먹고 싶다 해서 오예! 하고 방문했네용ㅎㅎ 아직 맛은 보지 않았으나 후기들만 봐도 평이 정말 좋아 벌써 만족스럽네요 크크- 맛있으면 자주 방문할게요 사장님 :)
                                              review
0  2호선을 이용할 때면 상왕십리역에서 내려 버스를 타곤 하는데, 그럴 때마다 눈에 쏘...


---

### 데이터셋1의 모든 가게에 대해 크롤링 적용

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("../data/raw/big_data_set1_f.csv", encoding='cp949')
print(df.head)

<bound method NDFrame.head of      ENCODED_MCT           MCT_BSE_AR        MCT_NM MCT_BRD_NUM  \
0     16184E93D9           서울 성동구 마장동          성우**         NaN   
1     4D039EA8B7           서울 성동구 마장동          대보**         NaN   
2     0074C4990A           서울 성동구 마장동          대용**         NaN   
3     68308F2746           서울 성동구 마장동          통일**         NaN   
4     4117EDDE9C           서울 성동구 마장동          한울**         NaN   
...          ...                  ...           ...         ...   
4180  7FCF23E6F3    서울특별시 성동구 마장로 137  아이**********         NaN   
4181  9743B78531      서울 성동구 독서당로 223          어반**         NaN   
4182  C2261977A8  서울특별시 성동구 마장로35길 76          한성**         NaN   
4183  F1C69918DF      서울 성동구 왕십리로 410          국수**   한식-국수/만두3   
4184  58B8C943BF     서울특별시 성동구 매봉길 50      아이******         NaN   

     MCT_SIGUNGU_NM HPSN_MCT_ZCD_NM HPSN_MCT_BZN_CD_NM     ARE_D  MCT_ME_D  
0            서울 성동구             축산물                마장동  20130320       NaN  
1          

df (가맹점 데이터)에서
앞의 5개 행만 선택 →
각 행마다

주소(MCT_BSE_AR) + 업종(HPSN_MCT_ZCD_NM)으로 네이버지도 검색

검색 결과 중에서
가게명(MCT_NM)의 마스킹 규칙을 반영하여
실제 이름과 일치하는 가게를 찾고

그 가게의 리뷰 5개까지 크롤링

결과를 DataFrame으로 저장

In [28]:
import time, re, difflib, pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ===== 설정 =====
LEN_TOL = 0
MAX_PAGES = 5
MAX_REVIEWS = 5  # 리뷰 5개 수집

# ===== 문자열 관련 유틸 =====
def mask_prefix_and_len(masked: str):
    return masked.split('*')[0], len(masked)

def normalize_name(s: str) -> str:
    s = re.sub(r"\([^)]*\)", "", s)
    s = re.sub(r"[\s/\-|·]", "", s)
    return s.strip()

# ===== 후보 중 최적 후보 선택 =====
def select_best_match(candidates, prefix, target_len):
    if not candidates:
        return None
    filtered = [c for c in candidates if normalize_name(c["name"]).startswith(prefix)]
    if not filtered:
        return None

    best, best_score = None, -1
    for c in filtered:
        clean = normalize_name(c["name"])
        len_diff = abs(len(clean) - target_len)
        sim = difflib.SequenceMatcher(None, clean, prefix).ratio()
        score = sim * 0.8 + (1 - len_diff / max(target_len, 1)) * 0.2
        if score > best_score:
            best_score = score
            best = c
    return best

# ===== 네이버 지도 제어 =====
def goto_home(driver, wait):
    driver.get("https://map.naver.com/p")
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "input_search")))

def clear_search_box(driver, wait):
    sb = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "input_search")))
    sb.click()
    sb.send_keys(Keys.CONTROL, 'a')
    sb.send_keys(Keys.DELETE)
    driver.execute_script("""
        const el = document.querySelector('.input_search');
        if (el) { el.value=''; el.dispatchEvent(new Event('input',{bubbles:true})); }
    """)
    return sb

def submit_query(driver, wait, query):
    driver.switch_to.default_content()
    sb = clear_search_box(driver, wait)
    sb.send_keys(query)
    val = sb.get_attribute("value").strip()

    if not val:
        print("⚠️ 검색어 비어 있음:", query)
        return False

    sb.send_keys(Keys.ENTER)
    time.sleep(1.2)

    try:
        driver.switch_to.default_content()
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
        time.sleep(0.8)
        return True
    except Exception:
        print("⚠️ 검색 iframe 로드 실패:", query)
        return False

def scroll_current_page(driver, loops=15, step_sleep=0.2):
    for _ in range(loops):
        prev = driver.execute_script("const c=document.querySelector('#_pcmap_list_scroll_container'); return c?c.scrollTop:0;")
        driver.execute_script("const c=document.querySelector('#_pcmap_list_scroll_container'); if(c) c.scrollTop = c.scrollTop + c.clientHeight*0.9;")
        time.sleep(step_sleep)
        cur = driver.execute_script("const c=document.querySelector('#_pcmap_list_scroll_container'); return c?c.scrollTop:0;")
        if cur == prev: break

def get_visible_candidates(driver):
    script = """
    const out = [];
    const root = document.querySelector('#_pcmap_list_scroll_container');
    if (!root) return out;

    const links = root.querySelectorAll('.place_bluelink');
    links.forEach(el=>{
        const name = el.textContent?.trim();
        const href = el.getAttribute('href') || '';
        if(name){
            out.push({name: name, href: href});
        }
    });
    return out;
    """
    try:
        return driver.execute_script(script) or []
    except Exception:
        return []

def go_next_page(driver, wait):
    cur_page = driver.execute_script("""
        const cur = document.querySelector('.zRM9F a.mBN2s.qxokY');
        return cur ? parseInt(cur.textContent.trim()) : null;
    """)
    is_disabled = driver.execute_script("""
        const next = document.querySelector('.zRM9F a.eUTV2');
        return next ? next.getAttribute('aria-disabled') === 'true' : false;
    """)
    if is_disabled: return False
    driver.execute_script("""
        const next = document.querySelector('.zRM9F a.eUTV2[aria-disabled="false"]');
        if (next) next.click();
    """)
    for _ in range(30):
        time.sleep(0.3)
        new_page = driver.execute_script("""
            const cur = document.querySelector('.zRM9F a.mBN2s.qxokY');
            return cur ? parseInt(cur.textContent.trim()) : null;
        """)
        if cur_page is not None and new_page is not None and new_page != cur_page:
            return True
    return False


# ===== 1️⃣ 매칭 단계 =====
def run_match_with_links(df):
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 15)

    results = []
    try:
        goto_home(driver, wait)
        total = len(df)
        print(f"\n🚀 전체 {total}개 가맹점 매칭 시작...")

        for i, row in df.iterrows():
            addr = str(row["MCT_BSE_AR"]).strip()
            cate = str(row["HPSN_MCT_ZCD_NM"]).strip()
            masked = str(row["MCT_NM"]).strip()
            full_name = masked.replace('*', '')
            prefix, tlen = mask_prefix_and_len(masked)
            query = f"{addr} {cate} {full_name}"

            print(f"\n[{i+1}/{total}] 검색어: {query}")

            if not submit_query(driver, wait, query):
                results.append({"MCT_NM": masked, "검색어": query, "결과": "검색실패", "매칭명": None, "매칭링크": None})
                continue

            no_result = driver.execute_script("""
                const el = document.querySelector('.empty_text, .no_result');
                return el && el.textContent.includes('조건에 맞는 업체가 없습니다');
            """)
            if no_result:
                print("🚫 조건에 맞는 업체 없음.")
                results.append({"MCT_NM": masked, "검색어": query, "결과": "조건없음", "매칭명": None, "매칭링크": None})
                continue

            li_count = driver.execute_script("return document.querySelectorAll('#_pcmap_list_scroll_container li').length;")
            if li_count == 0:
                print("⚠️ 검색 결과 없음 (li 비어있음).")
                results.append({"MCT_NM": masked, "검색어": query, "결과": "검색결과없음", "매칭명": None, "매칭링크": None})
                continue

            matched = None
            page_counter = 0
            while True:
                page_counter += 1
                scroll_current_page(driver)
                candidates = get_visible_candidates(driver)
                names = [c["name"] for c in candidates]
                print(f"  - {page_counter}페이지, 후보 {len(names)}개: {names[:3]}...")

                found = select_best_match(candidates, prefix, tlen)
                if found:
                    print(f"✅ 일치 가게: {found['name']}")
                    results.append({
                        "MCT_NM": masked, "검색어": query,
                        "결과": "일치", "매칭명": found["name"], "매칭링크": found["href"]
                    })
                    break

                if not go_next_page(driver, wait):
                    print("🚫 더 이상 페이지 없음.")
                    results.append({
                        "MCT_NM": masked, "검색어": query,
                        "결과": "불일치", "매칭명": None, "매칭링크": None
                    })
                    break

                if page_counter >= MAX_PAGES:
                    print("⚠️ 페이지 상한 도달.")
                    results.append({
                        "MCT_NM": masked, "검색어": query,
                        "결과": "불일치(상한)", "매칭명": None, "매칭링크": None
                    })
                    break
    finally:
        driver.quit()

    result_df = pd.DataFrame(results)
    print(f"\n✅ 매칭 완료 ({len(result_df)}개).")
    return result_df


# ===== 2️⃣ 리뷰 수집 단계 =====
def crawl_reviews_from_links(df_matched):
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 15)

    reviews = []
    try:
        total = len(df_matched)
        print(f"\n🧾 리뷰 수집 시작 ({total}개 가게)...")

        for i, row in df_matched.iterrows():
            name = row["매칭명"]
            link = row["매칭링크"]

            if pd.isna(link) or not link.strip():
                print(f"[{i+1}/{total}] ❌ 링크 없음: {row['MCT_NM']}")
                reviews.append({"MCT_NM": row["MCT_NM"], "매칭명": name, "리뷰": []})
                continue

            url = "https://map.naver.com" + link
            print(f"\n[{i+1}/{total}] ▶ {name} 리뷰 수집 중...")

            driver.get(url)
            time.sleep(2)

            try:
                # entryIframe 전환
                wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

                # 리뷰 탭 클릭
                review_tab = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@class,'_tab-menu')]//span[text()='리뷰']")))
                review_tab.click()
                time.sleep(2)

                # 리뷰 항목 추출
                elems = driver.find_elements(By.CSS_SELECTOR, "li.place_apply_pui div.pui__vn15t2 a, div.txt")
                texts = [e.text.strip() for e in elems if e.text.strip()]
                reviews_top5 = texts[:MAX_REVIEWS]

                print(f"✅ 리뷰 {len(reviews_top5)}개 수집.")
                reviews.append({"MCT_NM": row["MCT_NM"], "매칭명": name, "리뷰": reviews_top5})

            except Exception as e:
                print(f"⚠️ 리뷰 수집 실패: {name}")
                reviews.append({"MCT_NM": row["MCT_NM"], "매칭명": name, "리뷰": []})

            driver.switch_to.default_content()
    finally:
        driver.quit()

    reviews_df = pd.DataFrame(reviews)
    print(f"\n✅ 리뷰 수집 완료 ({len(reviews_df)}개).")
    return reviews_df



# 1️⃣ 가게 매칭 단계
result_df = run_match_with_links(df)

# 2️⃣ 리뷰 수집 단계
reviews_df = crawl_reviews_from_links(result_df)
display(reviews_df.head())



🚀 전체 4185개 가맹점 매칭 시작...

[1/4185] 검색어: 서울 성동구 마장동 축산물 성우
  - 1페이지, 후보 2개: ['성우축산정육점', '성우축산유통정육점']...
✅ 일치 가게: 성우축산정육점

[2/4185] 검색어: 서울 성동구 마장동 축산물 대보
  - 1페이지, 후보 1개: ['대보한우네이버페이톡톡정육점']...
✅ 일치 가게: 대보한우네이버페이톡톡정육점

[3/4185] 검색어: 서울 성동구 마장동 축산물 대용
  - 1페이지, 후보 4개: ['대용축산정육점', '대용축산정육점', '대용축산정육점']...
✅ 일치 가게: 대용축산정육점

[4/4185] 검색어: 서울 성동구 마장동 축산물 통일
⚠️ 검색 iframe 로드 실패: 서울 성동구 마장동 축산물 통일

[5/4185] 검색어: 서울 성동구 마장동 축산물 한울
  - 1페이지, 후보 1개: ['한울유통정육점']...
✅ 일치 가게: 한울유통정육점

[6/4185] 검색어: 서울 성동구 마장동 축산물 대광
  - 1페이지, 후보 3개: ['대광한우정육점', '대광상회육류', '대광축산정육점']...
✅ 일치 가게: 대광상회육류

[7/4185] 검색어: 서울 성동구 마장동 식품 제조 미림
⚠️ 검색 iframe 로드 실패: 서울 성동구 마장동 식품 제조 미림

[8/4185] 검색어: 서울 성동구 마장동 한식-육류/고기 호남
  - 1페이지, 후보 2개: ['호남집네이버페이톡톡육류,고기요리', '호남곱창곱창,막창,양']...
✅ 일치 가게: 호남곱창곱창,막창,양

[9/4185] 검색어: 서울 성동구 사근동 한식-찌개/전골 고향
⚠️ 검색 iframe 로드 실패: 서울 성동구 사근동 한식-찌개/전골 고향

[10/4185] 검색어: 서울 성동구 송정동 중식당 북
⚠️ 검색 iframe 로드 실패: 서울 성동구 송정동 중식당 북

[11/4185] 검색어: 서울 성동구 용답동 식료품 좋은
  - 1페이지, 후보 2개: ['좋은날푸드 서울본점전통식품', '곰탱이커피네이버페이커피

TimeoutException: Message: script timeout
  (Session info: chrome=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x00C3C113+48259]
	(No symbol) [0x00BCCA41]
	(No symbol) [0x00AC08A3]
	(No symbol) [0x00B3A533]
	(No symbol) [0x00B20D9C]
	(No symbol) [0x00B39B9C]
	(No symbol) [0x00B20B36]
	(No symbol) [0x00AF570D]
	(No symbol) [0x00AF62CD]
	GetHandleVerifier [0x00EF65A3+2908435]
	GetHandleVerifier [0x00F33BBB+3159851]
	GetHandleVerifier [0x00CD50CB+674875]
	GetHandleVerifier [0x00CDB28C+699900]
	(No symbol) [0x00BD6244]
	(No symbol) [0x00BD2298]
	(No symbol) [0x00BD242C]
	(No symbol) [0x00BC4BB0]
	BaseThreadInitThunk [0x75DF5D49+25]
	RtlInitializeExceptionChain [0x7767D6DB+107]
	RtlGetAppContainerNamedObjectPath [0x7767D661+561]
